In [80]:
import requests
import csv 
from collections import Counter
from pandas import *

listoffiles=[]
listoffilmnames=[]
        
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False 
    
def csvgetter(user,password,*otherusernames):
    s=requests.session()  #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',} #send the login data
    s.post("https://www.icheckmovies.com/login/", data=login_data) #csv link 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostcheckednum=row[3]
        if is_number(mostcheckednum) is True:
            mostcheckednum=int(row[3])
        mostchecked.append(mostcheckednum)
        toplistnum=row[5]
        if is_number(toplistnum) is True:
            toplistnum=int(row[5])
        toplist.append(toplistnum)        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

def process_club_list(user_list,others):
    merge_frames = others.merge(user_list,on=['title','most_checked'],how='left')
    not_same_titles = merge_frames[merge_frames.top_lists_y.isnull()]
    del not_same_titles['top_lists_y']
    not_same_titles.columns=['most_checked', 'title', 'top_lists']
    return(not_same_titles)

def isinlist(fromlist,tolist):
    fromlist=fromlist['title'].tolist()
    tolist=tolist['title'].tolist()
    if set(fromlist).issubset(set(tolist)):
        return True
    else:
        return False
    
def iseven(number):
    if number %2==0:
        return True
    else:
        return False

def lookupinlist(newtitle):
    if newframe.loc[newframe['title'] == newtitle]:
        return True
    else:
        return False

def final_table(club_list,length,method):
    by_checked=club_list.sort(['most_checked'],ascending=False)
    by_toplists=club_list.sort(['top_lists'],ascending=False)
    finaldata=DataFrame(columns=('most_checked','title','top_lists'))
    if method == 2:
        return by_toplists #Remove the most_checked element to filter out the trash for 2nd iteration
    elif method == 1:
        for counter in range(0, length):
            if iseven(counter) is True:
                rec = by_checked.iloc[[counter/2]]
                if counter == 0:
                    finaldata=finaldata.append(rec,ignore_index=True)
                else:
                    if isinlist(rec,finaldata) is True:
                        continue
                    else:
                        finaldata=finaldata.append(rec,ignore_index=True)
            else:
                rec = by_toplists.iloc[[(counter-1)/2]]
                if isinlist(rec,finaldata) is True:
                    continue
                else:
                    finaldata=finaldata.append(rec,ignore_index=True)
        return finaldata
    else:
        return finaldata


def Listmaker_pro(usr1,out,method,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    pw=open('password.txt','r')
    password=pw.read()
    csvgetter(username1,password,username2,username3)
    user1_list=listmaker(username1)
    user2_list=listmaker(username2)
    user3_list=listmaker(username3)
    combo_1_2=user1_list.append(user2_list)
    combo_1_3=user1_list.append(user3_list)
    combo_2_3=user2_list.append(user3_list)
    if method==1: #Original club list, with films both others had seen
        seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]
        seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]
        seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True]
    else: #For new list, where at least one other has seen.
        seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==False]
        seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==False]
        seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==False]
    club1_list=process_club_list(user1_list,seenby_2_3) 
    club2_list=process_club_list(user2_list,seenby_1_3)
    club3_list=process_club_list(user3_list,seenby_1_2)
    if out == 2:
        output=final_table(club2_list,20,method)
    elif out == 3:
        output=final_table(club3_list,20,method)
    else:
        output=final_table(club1_list,20,method)        
    return output

In [81]:
listus=Listmaker_pro('Haessar',1,2,'hermesmovies','James88')
listus

,most_checked,title,top_lists
1047,34941,Vertigo,38
735,46056,Casablanca,32
742,40600,Dr. Strangelove or: How I Learned to Stop Worr...,31
757,41640,12 Angry Men,27
1048,37652,Rear Window,26
1046,28386,North by Northwest,26
654,21950,Brazil,20
1042,42352,The Sound of Music,20
454,56841,Léon,18
642,40054,To Kill a Mockingbird,18


In [74]:
nus=listus
check=nus.to_html()
newone=check.encode('utf-8')


In [73]:
emailpw=open('emailpw.txt','r')
pwdmail=emailpw.read()

import smtplib
from email.mime.text import MIMEText 

msg=MIMEText(newone, 'html')
fromaddr = 'ICMmovielist@gmail.com'
toaddrs  = '@gmail.com'


username = 'ICMmovielist'

server = smtplib.SMTP('smtp.gmail.com:587')
server.starttls()
server.login(username,pwdmail)
server.sendmail(fromaddr, toaddrs, msg.as_string())
server.quit() 

SMTPRecipientsRefused: {'@gmail.com': (555, '5.5.2 Syntax error. g14sm24755823wjs.47 - gsmtp')}

In [29]:
test=open('email.txt','r')
ntest=test.read()

In [75]:
print(ntest)

NameError: name 'ntest' is not defined